## Prepare input structures

We first start by preparing the input ASE database. In the subfolder `structures` there are 9 example MOF structures. `save_to_db` is an convenient function to conver CIF files into one db.

In [1]:
from dotenv import load_dotenv
load_dotenv()

import glob
from ase.io import read
from mlip_arena.tasks.mof.input import save_to_db

files = glob.glob('structures/*.cif')

atoms_list = []

for file in files:
    atoms = read(file)
    atoms_list.append(atoms)

save_to_db(
    atoms_list=atoms_list,
    db_path='mofs.db'
)

2024-12-22 01:08:13.317 | INFO     | mlip_arena.tasks.mof:<module>:6 - 
The module 'mlip_arena.tasks.mof' is adapted from the repository: https://github.com/hspark1212/DAC-SIM. 
By using this module, you agree to the terms and conditions specified in the following license: 

https://github.com/hspark1212/DAC-SIM/blob/main/LICENSE

Additionally, please ensure proper attribution by citing the reference: 

Lim, Y., Park, H., Walsh, A., & Kim, J. (2024). Accelerating CO₂ Direct Air Capture Screening for Metal-Organic Frameworks with a Transferable Machine Learning Force Field.

A local copy of the LICENSE file can be found at: /pscratch/sd/c/cyrusyc/mlip-arena/mlip_arena/tasks/mof/LICENSE.



No module named 'deepmd'


/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/ase/io/cif.py:408: UserWarning: crystal system 'triclinic' is not interpreted for space group Spacegroup(1, setting=1). This may result in wrong setting!
  warnings.warn(
2024-12-22 01:08:15.893 | INFO     | mlip_arena.tasks.mof.input:save_to_db:45 - mofs.db uploaded to atomind/mlip-arena/mof


PosixPath('mofs.db')

## Demo: MOF benchmark workflow

In [1]:
from ase.build import molecule
from mlip_arena.models import MLIPEnum
from mlip_arena.tasks.mof.flow import run as MOF

@flow
def benchmark_one():
    results = []
    for model in MLIPEnum:
        if model.name != "MACE-MP(M)":
            continue # skip all models other than MACE-MP
        for atoms in tqdm(get_atoms_from_db('mofs.db')):
            result = widom_insertion(
                atoms, 
                molecule('CO2'),
                calculator_name=model.name,
            )
            results.append(result)
            break # only test on first MOF
    return [r for r in results]

No module named 'deepmd'


15:57:19.295 | WARNING | MDAnalysis.coordinates.AMBER - netCDF4 is not available. Writing AMBER ncdf files will be slow.

In [10]:
result = benchmark_one()
result[0]['henry_coefficient']

[2.3617881373330576e-06, 2.5985147795206755e-06]

## Run workflow

In [ ]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
from prefect_dask import DaskTaskRunner

from ase.build import molecule
from mlip_arena.models import MLIPEnum
from mlip_arena.tasks.mof.flow import run as MOF

MOF()